In [ ]:
import tensorflow_addons as tfa
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
from keras import backend as K
tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import socket
import importlib
import os
import sys
import datetime
import time 


BASE_DIR ='../input/vr3d-pts-bordeau'
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'logs'))
import tf_util

MODEL = importlib.import_module('pointnet_seg') # import network module
MAX_EPOCH = 21
NUM_POINT = 4096  # help='Point Number [256/512/1024/2048/4096] [default: 1024]
MAX_NUM_POINT = 4096
BATCH_SIZE = 32
NUM_CLASSES = 2 #2 #13 #40

BASE_LEARNING_RATE = 0.001#0.001
DECAY_STEP = 2000
DECAY_RATE = 0.3 #0.3

def get_learning_rate_schedule():
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        BASE_LEARNING_RATE,  # Initial learning rate
        DECAY_STEP,          # Decay step.
        DECAY_RATE,          # Decay rate.
        staircase=True)
    return learning_rate

def Dice_loss(y_true,y_pred): 
    # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / denominator

def DiceLoss(y_true, y_pred):
        # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * K.sum(y_true * y_pred)
    denominator = K.sum(y_true + y_pred)
    return 1 - numerator / denominator
    

def pixel_accuracy(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return (tp )/(tp+fp)  #(tp )/(tp+fp) pourcentage de la valeur positive prèdit sont vraie

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

def IOU(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return 1 -(tp )/(tp+fp+fn) # retourner en tant que loss

def train():
    with tf.device("gpu:0"):
            model = MODEL.get_model((None, 7), NUM_CLASSES)
            model.summary()
            learning_rate = get_learning_rate_schedule()
            # optimizer = tf.keras.optimizers.Adam(0.01)
            optimizer = tf.keras.optimizers.Adam(learning_rate)
        
            # initialize Dataset
            PointCloudProvider.initialize_dataset()
        
            print('\nput in generator...')
            generator_training = PointCloudProvider('train', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
            generator_validation = PointCloudProvider('test', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
            print('\ntraining...')
            
            log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            
            loss = tf.keras.losses.sparse_categorical_crossentropy
            model.compile(optimizer=optimizer,
                  loss = loss,
                  #loss = DiceLoss,
                  metrics = [IOU,'accuracy']
                  # metrics=["sparse_categorical_accuracy"]
                  #metrics=['accuracy']
                            )

            model.fit(generator_training, 
                      #validation_data=generator_validation,
                      steps_per_epoch=len(generator_training),
                      #validation_steps=len(generator_validation),
                      epochs=MAX_EPOCH,use_multiprocessing=False,
                      callbacks = [tensorboard_callback]
                     )
            
            print('\nsave model...')
            model.save_weights("/kaggle/working/Trained_ep_21_vr3d-pts_bordeau_"+datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")+"_Down_sample_bordeau_elevation_filtre.h5")
            print('\nFinished...')
            
            
if __name__ == "__main__":
    train()

Features Importantes

Zip log_file & download

In [ ]:
import os
from zipfile import ZipFile
zipO = ZipFile('Tensorboard_total.zip','w')
for i in range(2,13,40):
    for j in [20,28,32]:
        zipObj = ZipFile('Tensorboard'+str(i)+'-'+str(j)+'.zip', 'w')
        zipObj.write('./logs_'+str(i)+'-'+str(j))

        for dirname, _, filenames in os.walk('./logs_'+str(i)+'-'+str(j)):
            for filename in filenames:
                zipObj.write(os.path.join(dirname, filename))        
        zipObj.close()
        zipO.write('Tensorboard'+str(i)+'-'+str(j)+'.zip')
zipO.close()


In [ ]:
import os
from zipfile import ZipFile
zipObj = ZipFile('Tensorboard_mergeMas.zip','w')
zipObj.write('./logs')
for dirname, _, filenames in os.walk('./logs'):
    for filename in filenames:
        zipObj.write(os.path.join(dirname, filename))        
zipObj.close()

In [ ]:
import os
from zipfile import ZipFile
zipObj = ZipFile('tensorboard_pts.zip','w')
zipObj.write('./logs')
for dirname, _, filenames in os.walk('./logs'):
    for filename in filenames:
        zipObj.write(os.path.join(dirname, filename))        
zipObj.close()

In [ ]:
import os,glob
from zipfile import ZipFile
zipObj = ZipFile('model_to2.zip','w')
_path = './'
os.chdir(_path)
for file in glob.glob('T*.h5'):
    zipObj.write(file)        
zipObj.close()

In [ ]:
! wget --no-check-certificate https://github.com/RobinBaumann/pointnet/blob/master/tf2/train.py

In [ ]:
! unzip shapenet_part_seg_hdf5_data.zip

In [ ]:
! zip  down-sample-bordeau-elevation-filtre.zip

In [ ]:
import h5py
path = './hdf5_data/ply_data_train4.h5'
file_input = h5py.File(path,'r')


In [ ]:
import numpy as np

In [ ]:
file_input['data'][:]

Run Custom loss

In [ ]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
BASE_DIR = '../input/vr3d-pts-bordeau'
# BASE_DIR = '../input/experience'
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)
DATA_DIR = os.path.join(BASE_DIR, "data/h5_data/")



class PointCloudProvider(Sequence):
    """
    Lazily load point clouds and annotations from filesystem and prepare it for model training.
    """

    def __init__(self, mode='train', batch_size=32, n_classes=2, sample_size=4096, task="seg"): #task="classification"):
        """
        Instantiate a data provider instance for point cloud data.
        Args:
            dataset: pandas DataFrame containing
            n_classes: The number of different cthe index to the files (train or test set)
            batch_size: the desired batch sizelasses (needed for one-hot encoding of labels)
            sample_size: the amount of points to sample per instance.
            task: string denoting the tasks for which the data is to be loaded. Either "classification" (default) or "segmentaion".
        """
        self.datasets = {
            'train': 'train_VDS_7.h5',
            'test': 'test_VDS_7.h5'
        }
        self.mode = mode
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.sample_size = sample_size
        self.task = task

        self.indices = np.arange(0,len(h5py.File(self.datasets[self.mode], 'r')['data']), 1)
        print( 'Indice = ', self.indices )

        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil((len(self.indices)/self.batch_size)))

    def __getitem__(self, index):
        """Generate one batch of data."""
        batch_indices = self.indices[index * self.batch_size: (index + 1) * self.batch_size]
        #print('Generated batch_indices = ', batch_indices.shape)
        mask = np.zeros_like(self.indices).astype(np.bool)
        mask[batch_indices] = True
        X = h5py.File(self.datasets[self.mode], 'r')['data'][mask, ...]
        #print('X',X.shape)
        y = h5py.File(self.datasets[self.mode], 'r')['label'][mask, ...]
        #print('y', y.shape, '\n')

        #return np.array(X), to_categorical(np.array(y), num_classes=self.n_classes)
        #self.rotate_point_clouds(np.array(X))
        return np.array(X), np.array(y)

    def sample_random_points(self, pc):
        r_idx = np.random.randint(pc.shape[1], size=self.sample_size)
        return np.take(pc, r_idx, axis=1)

    def on_epoch_end(self):
        """Shuffle training data, so batches are in different order"""
        np.random.shuffle(self.indices)

    def rotate_point_clouds(self, batch, rotation_angle_range=(-np.pi / 8, np.pi / 8)):
        """Rotate point cloud around y-axis (=up) by random angle"""
        for b in range(batch.shape[0]):
            phi = np.random.uniform(*rotation_angle_range)
            c, s = np.cos(phi), np.sin(phi)
            R = np.asarray([[c, 0, s],
                           [0, 1, 0],
                           [-s, 0, c]])
            shape_pc = batch[b, ...]
            batch[b, ...] = np.dot(shape_pc.reshape((-1, 3)), R)
        return batch

    def load_h5(self, h5_filename):
        f = h5py.File(h5_filename, 'r')
        data = f['data'][:]
        label = f['label'][:]
        return data, label

    def load_data_file(self, filename):
        if self.task == "classification":
            return self.load_h5(filename)
        else:
            return self.load_h5_data_label_seg(filename)

    def load_h5_data_label_seg(self, h5_filename):
        f = h5py.File(h5_filename, 'r')
        data = f['data'][:]
        label = f['label'][:]
        #seg = f['pid'][:]
        return data, label  #, seg

    @staticmethod
    def initialize_dataset():
        """
        Loads an index to all files and structures them.
        :param data_directory: directory containing the data files
        :param file_extension: extension of the data files
        :return: pandas dataframe containing an index to all files and a label index,
            mapping numerical label representations to label names.
        """

        print("[Provider]: Creating Virtual Dataset")

        #train_index = os.path.join(BASE_DIR, 'train_files.txt')
        #test_index = os.path.join(BASE_DIR, 'test_files.txt')
        
        train_index = '../input/vr3d-pts-bordeau/data/h5_data/train_files.txt'
        test_index = '../input/vr3d-pts-bordeau/data/h5_data/test_files.txt'

        # train_index = '../input/experience/data/h5_data/train_files.txt'
        # test_index = '../input/experience/data/h5_data/test_files.txt'
        
        train_files = [line.rstrip() for line in open(train_index)]
        test_files = [line.rstrip() for line in open(test_index)]
        
        def create_vds(files, prefix='train'):

            print('\nVDS creation...')
            out_size = 0
            for f in files:
                out_size += h5py.File(DATA_DIR+f, 'r')['data'].shape[0]

            print('Total h5 size : ', out_size) #(23585, 4096, 9)

            # Assemble virtual dataset
            point_layout = h5py.VirtualLayout(shape=(out_size, 4096, 7), dtype='uint8')
            label_layout = h5py.VirtualLayout(shape=(out_size, 4096, 1), dtype='uint8')
            print('\nPoint VirtualLayout = ', point_layout.shape)
            print('Label VirtualLayout = ', label_layout.shape)

            for i, f in enumerate(files):
                h5_path = DATA_DIR+f
                size = len(h5py.File(h5_path, 'r')['data'])
                start_idx = i*1000  # only the last chunk is smaller than 2048
                end_idx = start_idx + size

                print('\nput in VirtualSource...')
                vsource_points = h5py.VirtualSource(h5_path, 'data', shape=(size, 4096, 7), maxshape=(out_size, 4096, 7))
                vsource_label = h5py.VirtualSource(h5_path, 'label', shape=(size, 4096, 1), maxshape=(out_size, 4096, 1))
                print('vsource_points = ',vsource_points.shape,
                      'vsource_label = ',vsource_label.shape)

                point_layout[start_idx:end_idx, ...] = vsource_points
                label_layout[start_idx:end_idx, ...] = vsource_label
                print('point_layout = ', point_layout.shape,
                      'label_layout = ', label_layout.shape)
                print('In : layout[', start_idx,':',end_idx,'...]')

            # Add virtual dataset to output file$

            with h5py.File("{}_VDS_7.h5".format(prefix), 'w', libver='latest') as f:
                print('\n\tcreate_virtual_dataset for DATA...')
                f.create_virtual_dataset('data', point_layout)
                print('\n\t', point_layout.shape)

                print('\n\tcreate_virtual_dataset for LABEL...')
                f.create_virtual_dataset('label', label_layout)
                print('\n\t', label_layout.shape)

        create_vds(train_files, 'train')
        create_vds(test_files, 'test')

        print("[Provider] Created Virtual Dataset.")


#init = PointCloudProvider()
#init.initialize_dataset()

Pointcloud Baumann

In [ ]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
BASE_DIR = '../input/vr3d-baumann'
# BASE_DIR = '../input/experience'
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)
DATA_DIR = os.path.join(BASE_DIR, "data/h5_data/")



class PointCloudProvider(Sequence):
    """
    Lazily load point clouds and annotations from filesystem and prepare it for model training.
    """

    def __init__(self, mode='train', batch_size=32, n_classes=2, sample_size=2048, task="seg"): #task="classification"):
        """
        Instantiate a data provider instance for point cloud data.
        Args:
            dataset: pandas DataFrame containing
            n_classes: The number of different cthe index to the files (train or test set)
            batch_size: the desired batch sizelasses (needed for one-hot encoding of labels)
            sample_size: the amount of points to sample per instance.
            task: string denoting the tasks for which the data is to be loaded. Either "classification" (default) or "segmentaion".
        """
        self.datasets = {
            'train': 'train_VDS_7.h5',
            'test': 'test_VDS_7.h5'
        }
        self.mode = mode
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.sample_size = sample_size
        self.task = task

        self.indices = np.arange(0,len(h5py.File(self.datasets[self.mode], 'r')['data']), 1)
        print( 'Indice = ', self.indices )

        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil((len(self.indices)/self.batch_size)))

    def __getitem__(self, index):
        """Generate one batch of data."""
        batch_indices = self.indices[index * self.batch_size: (index + 1) * self.batch_size]
        #print('Generated batch_indices = ', batch_indices.shape)
        mask = np.zeros_like(self.indices).astype(np.bool)
        mask[batch_indices] = True
        X = h5py.File(self.datasets[self.mode], 'r')['data'][mask, ...]
        #print('X',X.shape)
        y = h5py.File(self.datasets[self.mode], 'r')['label'][mask, ...]
        #print('y', y.shape, '\n')

        #return np.array(X), to_categorical(np.array(y), num_classes=self.n_classes)
        #self.rotate_point_clouds(np.array(X))
        return np.array(X), np.array(y)

    def sample_random_points(self, pc):
        r_idx = np.random.randint(pc.shape[1], size=self.sample_size)
        return np.take(pc, r_idx, axis=1)

    def on_epoch_end(self):
        """Shuffle training data, so batches are in different order"""
        np.random.shuffle(self.indices)

    def rotate_point_clouds(self, batch, rotation_angle_range=(-np.pi / 8, np.pi / 8)):
        """Rotate point cloud around y-axis (=up) by random angle"""
        for b in range(batch.shape[0]):
            phi = np.random.uniform(*rotation_angle_range)
            c, s = np.cos(phi), np.sin(phi)
            R = np.asarray([[c, 0, s],
                           [0, 1, 0],
                           [-s, 0, c]])
            shape_pc = batch[b, ...]
            batch[b, ...] = np.dot(shape_pc.reshape((-1, 3)), R)
        return batch

    def load_h5(self, h5_filename):
        f = h5py.File(h5_filename, 'r')
        data = f['data'][:]
        label = f['label'][:]
        return data, label

    def load_data_file(self, filename):
        if self.task == "classification":
            return self.load_h5(filename)
        else:
            return self.load_h5_data_label_seg(filename)

    def load_h5_data_label_seg(self, h5_filename):
        f = h5py.File(h5_filename, 'r')
        data = f['data'][:]
        label = f['label'][:]
        #seg = f['pid'][:]
        return data, label  #, seg

    @staticmethod
    def initialize_dataset():
        """
        Loads an index to all files and structures them.
        :param data_directory: directory containing the data files
        :param file_extension: extension of the data files
        :return: pandas dataframe containing an index to all files and a label index,
            mapping numerical label representations to label names.
        """

        print("[Provider]: Creating Virtual Dataset")

        #train_index = os.path.join(BASE_DIR, 'train_files.txt')
        #test_index = os.path.join(BASE_DIR, 'test_files.txt')
        
        train_index = '../input/vr3d-baumann/data/h5_data/train_files.txt'
        test_index = '../input/vr3d-baumann/data/h5_data/test_files.txt'

        # train_index = '../input/experience/data/h5_data/train_files.txt'
        # test_index = '../input/experience/data/h5_data/test_files.txt'
        
        train_files = [line.rstrip() for line in open(train_index)]
        test_files = [line.rstrip() for line in open(test_index)]
        
        def create_vds(files, prefix='train'):

            print('\nVDS creation...')
            out_size = 0
            for f in files:
                out_size += h5py.File(DATA_DIR+f, 'r')['data'].shape[0]

            print('Total h5 size : ', out_size) #(23585, 4096, 9)

            # Assemble virtual dataset
            point_layout = h5py.VirtualLayout(shape=(out_size, 2048, 3), dtype='uint8')
            label_layout = h5py.VirtualLayout(shape=(out_size, 2048, 1), dtype='uint8')
            print('\nPoint VirtualLayout = ', point_layout.shape)
            print('Label VirtualLayout = ', label_layout.shape)

            for i, f in enumerate(files):
                h5_path = DATA_DIR+f
                size = len(h5py.File(h5_path, 'r')['data'])
                start_idx = i*1000  # only the last chunk is smaller than 2048
                end_idx = start_idx + size

                print('\nput in VirtualSource...')
                vsource_points = h5py.VirtualSource(h5_path, 'data', shape=(size, 2048, 3), maxshape=(out_size, 2048, 3))
                vsource_label = h5py.VirtualSource(h5_path, 'label', shape=(size, 2048, 1), maxshape=(out_size, 2048, 1))
                print('vsource_points = ',vsource_points.shape,
                      'vsource_label = ',vsource_label.shape)

                point_layout[start_idx:end_idx, ...] = vsource_points
                label_layout[start_idx:end_idx, ...] = vsource_label
                print('point_layout = ', point_layout.shape,
                      'label_layout = ', label_layout.shape)
                print('In : layout[', start_idx,':',end_idx,'...]')

            # Add virtual dataset to output file$

            with h5py.File("{}_VDS_7.h5".format(prefix), 'w', libver='latest') as f:
                print('\n\tcreate_virtual_dataset for DATA...')
                f.create_virtual_dataset('data', point_layout)
                print('\n\t', point_layout.shape)

                print('\n\tcreate_virtual_dataset for LABEL...')
                f.create_virtual_dataset('label', label_layout)
                print('\n\t', label_layout.shape)

        create_vds(train_files, 'train')
        create_vds(test_files, 'test')

        print("[Provider] Created Virtual Dataset.")


#init = PointCloudProvider()
#init.initialize_dataset()

Model with gridsearchcv

In [ ]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import keras.backend as K
from tensorboard.plugins.hparams import api as hp
tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import socket
import importlib
import os
import sys
import datetime
import time 

BASE_DIR = '../input/vr3d-calibre-intensite'
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'logs'))
#from provider2 import PointCloudProvider
import tf_util

MODEL = importlib.import_module('pointnet_seg') 

#MAX_EPOCH = 15
# Set the number of points to sample and batch size and parse the dataset.
NUM_POINT = 4096  # help='Point Number [256/512/1024/2048/4096] [default: 1024]
MAX_NUM_POINT = 4096 # default 4096
BATCH_SIZE = 32
NUM_CLASSES = 2 #13 #40

BASE_LEARNING_RATE = 0.001 # à descendre
DECAY_STEP = 2000 # à monter
DECAY_RATE = 0.5 # à desc 

def get_learning_rate_schedule(instanc):
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        BASE_LEARNING_RATE*10**(-instanc),  # Initial learning rate
        DECAY_STEP,          # Decay step.
        DECAY_RATE-0.1*instanc,          # Decay rate.
        staircase=True)
    return learning_rate

def get_opt(instanc):
    learning_rate = get_learning_rate_schedule(instanc)
    # learning_rate = 0.0001
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    return optimizer


def Dice_loss(y_true,y_pred): 
    #custom_loss=kb.square(y_actual-y_pred)
    #https://arxiv.org/pdf/2003.07311.pdf (p3)
   
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

def IOU_LOSS(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return 1 -(tp )/(tp+fp+fn) # retourner en tant que loss

def pixel_accuracy(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return (tp )/(tp+fp)


def model_m(model,instanc,MAX_EPOCH):
    PointCloudProvider.initialize_dataset()
    print('\nput in generator...')
        
    generator_training = PointCloudProvider('train', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
    generator_validation = PointCloudProvider('test', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
        
    print('\ntraining...')
    print(len(generator_training))
    model.compile(optimizer=get_opt(instanc),
                  loss = tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=[pixel_accuracy,IOU_LOSS,'accuracy'])
    
    log_dir = "logs/logs_"+str(instanc)+"-"+str(MAX_NUM_POINT)+"/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) 
    model.fit(generator_training,
                        steps_per_epoch=len(generator_training),
                        epochs=MAX_EPOCH,use_multiprocessing=False,callbacks =[tensorboard_callback]
             )
    model.save_weights("/kaggle/working/Trained_ep15_"+str(instanc)+"-"+str(MAX_NUM_POINT)+"_Down_sample_bordeau_elevation_filtre.h5")
    
    print('*'*20,'resulta','*'*20)
    print(f'Base_learning rate :{0.0001*10**(-instanc)}\n')
    print(f'Decay_rate:{0.5-0.1*instanc}\n')
    print(f'Batch_size:{MAX_NUM_POINT}\n')
    print(f'num_class:{instanc}\n')
    print('*'*20,'resulta','*'*20)
    
def train():
    with tf.device("gpu:0"):
        max_score = 0
        for i in range(1,3):
            for j in [18,20,22,23]:#batch_size
                model = MODEL.get_model((None, 7), NUM_CLASSES) 
                model_m(model,i,j)
                # result,instanc = model_metric(model,i,j)
                # max_score = max(result,max_score)
        # print(f'the best IoU score is: {max_score}')
if __name__ == "__main__":
    train()

In [ ]:
import tensorflow_addons as tfa
import argparse
import math

import h5py
import numpy as np
import tensorflow as tf
from keras import backend as K
from tensorflow.keras import backend as K, Model
from tensorflow.keras.layers import Input, Lambda, concatenate
tf.compat.v1.disable_eager_execution()
from keras.models import Sequential
#tf.compat.v1.enable_eager_execution()
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import socket
import importlib
import os
import sys
import datetime
import time 


BASE_DIR ='../input/down-sample-bordeau-elevation-filtre'
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'logs'))
import tf_util

MODEL = importlib.import_module('pointnet_seg') # import network module

MAX_EPOCH = 5
NUM_POINT = 4096  # help='Point Number [256/512/1024/2048/4096] [default: 1024]
MAX_NUM_POINT = 4096
BATCH_SIZE = 32
NUM_CLASSES = 2 #2 #13 #40

BASE_LEARNING_RATE = 0.001
DECAY_STEP = 20000
DECAY_RATE = 0.3

def get_learning_rate_schedule():
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        BASE_LEARNING_RATE,  # Initial learning rate
        DECAY_STEP,          # Decay step.
        DECAY_RATE,          # Decay rate.
        staircase=True)
    return learning_rate

def Dice_loss(y_true,y_pred): 
    # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / denominator

def DiceLoss(y_true, y_pred):
        # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * K.sum(y_true * y_pred)
    denominator = K.sum(y_true + y_pred)
    return 1 - numerator / denominator
    

def pixel_accuracy(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return (tp )/(tp+fp)  #(tp )/(tp+fp) pourcentage de la valeur positive prèdit sont vraie

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

def IOU(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return 1 -(tp )/(tp+fp+fn) # retourner en tant que loss

def train():
    with tf.device("gpu:0"):
            new_model = get_model((None, 7), NUM_CLASSES)
            #model = MODEL.get_model((4096, 7), NUM_CLASSES)
            new_model.summary()
            #inputs = Input((4096,7), name='Input_cloud')
            # Reshape the output model
            #new_model = Sequential()
            #new_model.add(model)
            #new_model.add(tf.keras.layers.Flatten())
            #new_model.add(tf.keras.layers.Dense(2,activation='relu'))
            #new_model.add(tf.keras.layers.Reshape((-1,2)))
            #x = tf.keras.layers.Flatten()
            #outputs = tf.keras.layers.Dense(2, activation='relu')(x)# inputs and outputs
            #new_model = tf.keras.Model(inputs=model, outputs=outputs, name="test_for_shap")
            #new_model.summary()
            
            
            
            learning_rate = get_learning_rate_schedule()
            # optimizer = tf.keras.optimizers.Adam(0.01)
            optimizer = tf.keras.optimizers.Adam(learning_rate)
        
            # initialize Dataset
            PointCloudProvider.initialize_dataset()
        
            print('\nput in generator...')
            generator_training = PointCloudProvider('train', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
            generator_validation = PointCloudProvider('test', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
            print('\ntraining...')
            
            log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            
            loss = tf.keras.losses.sparse_categorical_crossentropy
            new_model.compile(optimizer=optimizer,
                              loss = loss,
                              #loss = DiceLoss,
                              #metrics = [pixel_accuracy,IOU,'accuracy']
                              # metrics=["sparse_categorical_accuracy"]
                              metrics=['accuracy']
                            )
            pred = new_model.predict(generator_training) 
            print(pred,pred.shape)
            new_model.save_weights("/kaggle/working/Trained_epSHAP_"+datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")+"_Down_sample_bordeau_elevation_filtre.h5")
            new_model.fit(generator_training, 
                      #validation_data=generator_validation,
                      steps_per_epoch=len(generator_training),
                      #validation_steps=len(generator_validation),
                      epochs=MAX_EPOCH,use_multiprocessing=False,
                      callbacks = [tensorboard_callback]
                     )
            
            print('\nsave model...')
            
            print('\nFinished...')
            
            
if __name__ == "__main__":
    train()

In [ ]:
import tensorflow_addons as tfa
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
from keras import backend as K
tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import socket
import importlib
import os
import sys
import datetime
import time 


BASE_DIR ='../input/vr3d-baumann'
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'logs'))
import tf_util

MODEL = importlib.import_module('pointnet_seg') # import network module

MAX_EPOCH = 30
NUM_POINT = 4096  # help='Point Number [256/512/1024/2048/4096] [default: 1024]
MAX_NUM_POINT = 2048
BATCH_SIZE = 32
NUM_CLASSES = 2 #2 #13 #40

BASE_LEARNING_RATE = 0.00001
DECAY_STEP = 2000
DECAY_RATE = 0.3

def get_learning_rate_schedule():
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        BASE_LEARNING_RATE,  # Initial learning rate
        DECAY_STEP,          # Decay step.
        DECAY_RATE,          # Decay rate.
        staircase=True)
    return learning_rate

def Dice_loss(y_true,y_pred): 
    # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / denominator

def DiceLoss(y_true, y_pred):
        # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * K.sum(y_true * y_pred)
    denominator = K.sum(y_true + y_pred)
    return 1 - numerator / denominator
    

def pixel_accuracy(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return (tp )/(tp+fp)  #(tp )/(tp+fp) pourcentage de la valeur positive prèdit sont vraie

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

def IOU(y_true,y_pred):
    tp = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1 - y_pred), 0, 1)))
    return 1 -(tp )/(tp+fp+fn) # retourner en tant que loss

def train():
    with tf.device("gpu:0"):
            model = MODEL.get_model((None, 3), NUM_CLASSES)
            model.summary()
            learning_rate = get_learning_rate_schedule()
            # optimizer = tf.keras.optimizers.Adam(0.01)
            optimizer = tf.keras.optimizers.Adam(learning_rate)
        
            # initialize Dataset
            PointCloudProvider.initialize_dataset()
        
            print('\nput in generator...')
            generator_training = PointCloudProvider('train', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
            generator_validation = PointCloudProvider('test', BATCH_SIZE, n_classes=NUM_CLASSES, sample_size=MAX_NUM_POINT)
            print('\ntraining...')
            
            log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            
            loss = tf.keras.losses.sparse_categorical_crossentropy
            model.compile(optimizer=optimizer,
                          loss=loss,
                          #metrics=['categorical_accuracy']
                          #loss = DiceLoss,
                          metrics = [pixel_accuracy,IOU,'accuracy']
                          #metrics=["sparse_categorical_accuracy"]
                          #metrics=['accuracy']
                            )

            model.fit(generator_training, 
                      #validation_data=generator_validation,
                      steps_per_epoch=len(generator_training),
                      #validation_steps=len(generator_validation),
                      epochs=MAX_EPOCH,use_multiprocessing=False,
                      callbacks = [tensorboard_callback]
                     )
            
            print('\nsave model...')
            model.save_weights("/kaggle/working/Trained_ep30BA_"+datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")+"_Down_sample_bordeau_elevation_filtre.h5")
            print('\nFinished...')
            
            
if __name__ == "__main__":
    train()